# Install Libraries:

In [ ]:
!pip install opencv-python
!pip install scenedetect


# Splitting the video

In [ ]:
#detect and split the video and but the result videos in the result folder
!scenedetect -i ../videos/video.mp4 detect-content split-video -o ../result

# Get the times of each scene

In [1]:
# function to get the times of each scene
from scenedetect import VideoManager
from scenedetect import SceneManager

# For content-aware scene detection:
from scenedetect.detectors import ContentDetector

def find_scenes(video_path, threshold=30.0):
    # Create our video & scene managers, then add the detector.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    # Improve processing speed by downscaling before processing.
    video_manager.set_downscale_factor()

    # Start the video manager and perform the scene detection.
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)

    # Each returned scene is a tuple of the (start, end) timecode.
    return scene_manager.get_scene_list()

In [ ]:
scenes = find_scenes("../Videos/video.mp4")
scenes


# Return image description from imagga

In [3]:
! pip install requests

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [10]:
#Not Working

import os
import json
import requests
import argparse



from create_index import API_KEY, API_SECRET, index_endpoint

DISTANCE_THRESHOLD = 1.4


def search_index(image_path):
    if not os.path.exists(image_path):
        print('Path is invalid:', image_path)
        return None

    search_result = None
    with open(image_path, 'rb') as image_file:
        response = requests.post(
            index_endpoint,
            params={'distance': DISTANCE_THRESHOLD},
            files={'image': image_file},
            auth=(API_KEY, API_SECRET))

    try:
        search_result = response.json()['result']
    except Exception as e:
        print('Exception occured during reading the search response')
        print(e, response.content, response.status_code)

    return search_result


def main():
    #parser = argparse.ArgumentParser()
    #parser.add_argument('image_path', help='Image to search for')

    #args = parser.parse_args()

    result = search_index("D:\\College\\2021-1\\Senior\\Style-based-Video-Editor\\images\\testimages\\DSC_0086.JPG")
    if not result:
        print('No result obtained')

    print('Search result:')
    print(json.dumps(result, indent=4))


if __name__ == '__main__':
    main()




ModuleNotFoundError: No module named 'create_index'

In [5]:
#Face Detection Working

import os
import requests
import time

API_KEY = 'acc_6143e9674b1c218'
API_SECRET = '91dcb4c50d92be863573c465d9136506'

API_ENDPOINT = 'https://api.imagga.com/v2'
CATEGORIZER = 'detections'  # The default general purpose categorizer
INDEX_NAME = 'similarity_tutorial_v1'

categorizer_endpoint = '%s/faces/%s' % (API_ENDPOINT, CATEGORIZER)
index_endpoint = ('%s/similar-images/categories/%s/%s' %
                  (API_ENDPOINT, CATEGORIZER, INDEX_NAME))
tickets_endpoint = '%s/tickets' % API_ENDPOINT

image_path="D:\\College\\2021-1\\Senior\\Style-based-Video-Editor\\images\\testimages\\DSC_0086.JPG"

response = requests.post(
        categorizer_endpoint,
        auth=(API_KEY, API_SECRET),
        files={'image': open(image_path, 'rb')},
        params={'image_url': 'friends',})

status = response.json()['status']

response.json()

{'result': {'faces': [{'attributes': [],
    'confidence': 99.9869842529297,
    'coordinates': {'height': 508,
     'width': 508,
     'xmax': 1141,
     'xmin': 633,
     'ymax': 1961,
     'ymin': 1453},
    'face_id': '',
    'landmarks': []},
   {'attributes': [],
    'confidence': 99.9829177856445,
    'coordinates': {'height': 440,
     'width': 440,
     'xmax': 4398,
     'xmin': 3958,
     'ymax': 1933,
     'ymin': 1493},
    'face_id': '',
    'landmarks': []},
   {'attributes': [],
    'confidence': 99.9795150756836,
    'coordinates': {'height': 463,
     'width': 463,
     'xmax': 3496,
     'xmin': 3033,
     'ymax': 1982,
     'ymin': 1519},
    'face_id': '',
    'landmarks': []},
   {'attributes': [],
    'confidence': 99.9623641967773,
    'coordinates': {'height': 460,
     'width': 460,
     'xmax': 2670,
     'xmin': 2210,
     'ymax': 2078,
     'ymin': 1618},
    'face_id': '',
    'landmarks': []}]},
 'status': {'text': '', 'type': 'success'}}

In [7]:
#Find similar images to a spacific image

import os
import requests
import time

API_KEY = 'acc_6143e9674b1c218'
API_SECRET = '91dcb4c50d92be863573c465d9136506'

API_ENDPOINT = 'https://api.imagga.com/v2'
CATEGORIZER = 'general_v3'  # The default general purpose categorizer
INDEX_NAME = 'similarity_tutorial_v1'

categorizer_endpoint = '%s/categories/%s' % (API_ENDPOINT, CATEGORIZER)
index_endpoint = ('%s/similar-images/categories/%s/%s' %
                  (API_ENDPOINT, CATEGORIZER, INDEX_NAME))
tickets_endpoint = '%s/tickets' % API_ENDPOINT

def feed_image(image_path):
    file_name = os.path.basename(image_path)
    image_name, extension = os.path.splitext(file_name)
    print(image_name)
    # Make the API call
    response = requests.post(
        categorizer_endpoint,
        auth=(API_KEY, API_SECRET),
        files={'image': open(image_path, 'rb')},
        params={'save_id': image_name, 'save_index': INDEX_NAME})

    # Check for errors
    try:
        status = response.json()['status']
        if status['type'] != 'success':
            print('API reported error:', status['text'])
            return False
    except Exception as e:
        print('Exception occured during handling the feeding response')
        print(e)
        return False

    return True


def train_index():
    ticket_id = ''

    # Make the API call
    response = requests.put(index_endpoint, auth=(API_KEY, API_SECRET))

    # Get the completion ticket
    try:
        ticket_id = response.json()['result']['ticket_id']
    except Exception as e:
        print('Exception occured when processing the train call response')
        print(e, response.content)

    return ticket_id

def is_resolved(ticket_id):
    resolved = False

    # Make the API call
    response = requests.get(
        '%s/%s' % (tickets_endpoint, ticket_id), auth=(API_KEY, API_SECRET))

    # Check for errors
    try:
        resolved = response.json()['result']['is_final']
    except Exception as e:
        print('Exception occured during the ticket status check')
        print(e)

    return resolved

def main():
    # Feed the visual search index with images
    successful_feeds = 0
    failed_feeds = 0

    for image in os.scandir('C:\\Users\\LENOVO\\Desktop\\senior\\imagga_similarity_api_usage_code\\index_creation_set'):
        print('At image %s: %s' % (successful_feeds, image.path))

        successful = feed_image(image.path)
        if successful:
            successful_feeds += 1
        else:
            failed_feeds += 1

    # Report
    if not successful_feeds:
        print('No images were fed successfully. Exiting')
        return

    print(
        '%s out of %s images fed successfully' %
        (successful_feeds, successful_feeds + failed_feeds))

    # Train the index
    ticket_id = train_index()

    if not ticket_id:
        print('No ticket id. Exiting')
        return

    # Wait for the training to complete
    time_started = time.time()
    while not is_resolved(ticket_id):
        t_passed = (time.time() - time_started) // 1000
        print(
            'Waiting for training to finish (time elapsed: %.1fs)' % t_passed)

        time.sleep(0.5)

    print('Training done.')


if __name__ == '__main__':
    main()



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\LENOVO\\Desktop\\senior\\imagga_similarity_api_usage_code\\index_creation_set'